In [1]:
pip install shiny

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 8.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.7/426.7 KB 11.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 1.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_e

In [2]:
import os, torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 2 to use
torch.set_num_threads(16)
# torch.cuda.set_per_process_memory_fraction(0.8, device=None)

In [3]:
# import preprocessing
import model

In [4]:
import pandas as pd
import json
import numpy as np

# CSV 파일 읽기
df_loaded = pd.read_csv('../ecg_data.csv')

# JSON 문자열을 dictionary로 변환하고, 리스트를 다시 numpy 배열로 복원
def list_to_numpy(d):
    return {k: np.array(v, dtype=np.float32) for k, v in d.items()}

df_loaded['WaveForm'] = df_loaded['WaveForm'].apply(json.loads)
df_loaded['WaveForm'] = df_loaded['WaveForm'].apply(list_to_numpy)

# 확인

In [5]:
# df = preprocessing.p()

In [6]:
df_loaded['Vent'] = df_loaded['Vent'].replace('1->0', '1')

In [7]:
df = df_loaded

In [1]:
from model_res import *
from model import preprocess_data

In [ ]:
def predict_from_xml(xml_file):
    # new_data = p(xml_file)
    new_data = xml_file
    # 사용 예시
    checkpoint_path = "/vit_best"
    vit = model_res.load_vit_model(checkpoint_path)

    # 3. 새로운 데이터에 대한 예측
    probabilities = model_res.predict_probabilities(vit, new_data)

    # 결과 출력
    for i, prob in enumerate(probabilities):
        print(f"Probability of class 1 = {prob:.4f}")
        print(f"Probability of class 0 = {1-prob:.4f}")
    return (prob,1-prob)

In [ ]:
new_data = preprocess_data(df)[:2]
predict_from_xml(new_data)

In [ ]:
pip install 

### 원래 xml로..ㅠ

In [1]:
from model_res import *
from model import preprocess_data

In [2]:
predict_from_xml('03812187_20040831_1506.xml')

2024-09-13 08:15:30.058527: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 08:16:29.748637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42800 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:45:00.0, compute capability: 8.6
2024-09-13 08:16:29.749580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 44718 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:48:00.0, compute capability: 8.6
2024-09-13 08:16:29.750143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

Probability of class 1 = 0.2018
Probability of class 0 = 0.7982


2024-09-13 08:16:44.083168: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(array([0.20183718], dtype=float32), array([0.7981628], dtype=float32))

In [33]:
from shiny import App, ui, reactive
from model_res import predict_from_xml
app_ui = ui.page_fluid(
    ui.h2("Predict Model Output from XML"),  # 애플리케이션 제목
    ui.row(
        ui.column(
            6,
            ui.panel_well(
                ui.h3("Step 1: Upload Your XML File"),
                ui.input_file("xml_file", "Choose an XML file", accept=[".xml"])
            )
        ),
        ui.column(
            6,
            ui.panel_well(
                ui.h3("Step 2: View Prediction Result"),
                ui.output_text_verbatim("output_prediction")
            )
        )
    ),
    ui.hr(),  # 구분선 추가
    ui.h5("This app uses a machine learning model to predict the output based on XML input.")
)

# 서버 로직 정의
def server(input, output, session):
    # 파일 업로드 감지
    @reactive.Effect
    @reactive.event(input.xml_file)
    def _():
        xml_info = input.xml_file()
        if xml_info is not None:
            # 파일 경로 받아오기
            xml_path = xml_info[0]["datapath"]

            # 모델로 예측 수행
            prediction = predict_from_xml(xml_path)

            # 예측 결과 출력
            output.output_prediction.set(f"Probability of 0: {prediction[0]}, Probability of 1: {prediction[1]}")

# Shiny 애플리케이션 생성
app = App(app_ui, server)


SyntaxError: invalid syntax (4011993584.py, line 1)